# FOOD AND BEVERAGE BUSINESS OPPORTUNITY SEEKER

This is an independent project centered around data science application in food and beverage industry. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 4>

1. <a href="#item1">Introduction</a>
    
2. <a href="#item2">Data</a>

</font>
</div>

## 1. Introduction

There is a businessman lives in the capital city of Indonesia, Jakarta. He wants to start a food and Beverage (FnB) business in the city where he is living in, but he doesn't know how to start. Without any experience in this kind of business, he comes to us and tells us about his plan. He hopes that you can help him to make a right decision. As a Data Scientist, you are challenged to support him by giving him best recommendation in order to build a sucessful FnB business.

This new businessman has two critical questions here, they are:
1. What kind of food and beverage business that he should start with?
2. Where in the city should he start this business?

## 2. Data

In order to answer those two questions, as a data scientist, we need to have data related to the food and beverage market in Jakarta. Data of existing restaurant or food court venues in Jakarta would be useful for our analysis. Foursquare has public API that can provide us the data.  

Beforehand, let us see the data that we can use from the Foursquare API. Foursquare has some account tiers for developers. Each tier has different available features. You can read the full documentation <a href="https://developer.foursquare.com/comparison">here</a> and <a href="https://developer.foursquare.com/docs">here</a>. Supposed that we only want to access the regular tier. You can see the available endpoint for this tier <a href="https://developer.foursquare.com/docs/api/endpoints">here</a>. In this tier, we can get the data of trending venues near the specific location with the most people currently checked in. We can use this endpoint to find most trending restaurant or food court venues in Jakarta. The data should contain:
+ ratings
+ likes
+ checkin counts
+ longitude
+ latitude
+ venue type
+ tips

These data can be used to answer the first question. We can find what kind of FnB business that is popular now in Jakarta. We can consider the data in determining what kind of FnB business that we want to begin with. But, looking at the current trending FnB venues is not enough for us to make a good decision. Let's take more data into consideration. Nearby venues from each popular FnB venues can also be useful. We can use it to find external factor of why these popular venues that we have got from previous step so popular. Is it because the easier transportation access, or located near shopping malls, or whatever you can think. This will help us to answer the second question. We can recommend the best location to put our restaurant in the city.   